In [51]:
import pandas as pd

In [52]:
df = pd.read_table('data/1.tsv')
df2 = pd.read_table('data/2.tsv')

In [53]:
df.pop('lensfiletime_utc')
df.head()

,time_kst,value
0,201605011300,1.24
1,201605011400,1.22
2,201605011500,1.20
3,201605011600,1.20
4,201605011700,1.20


In [54]:
df2.dropna(inplace=True)
df2.pop('lensfiletime_kst')
df2.head()

,targetdate12digit,class
0,2.016050e+11,0.0
1,2.016050e+11,0.0
2,2.016050e+11,1.0
3,2.016050e+11,1.0
4,2.016050e+11,1.0


In [55]:
df = df.join(df2.set_index('targetdate12digit'), on='time_kst')
df.head()

,time_kst,value,class
0,201605011300,1.24,0.0
1,201605011400,1.22,NaN
2,201605011500,1.20,NaN
3,201605011600,1.20,NaN
4,201605011700,1.20,NaN


In [56]:
# 시간 순으로 정렬
df.sort_values(by='time_kst', inplace=True)

In [57]:
# 결측값 채우기
df.fillna(method='ffill', inplace=True)
df.head()

,time_kst,value,class
0,201605011300,1.24,0.0
1,201605011400,1.22,0.0
2,201605011500,1.20,0.0
3,201605011600,1.20,0.0
4,201605011700,1.20,0.0


In [58]:
df['time'] = pd.to_datetime(df['time_kst'], format='%Y%m%d%H%M%S')
df.drop('time_kst', axis=1, inplace=True)

In [59]:
df.head()

,value,class,time
0,1.24,0.0,2016-05-01 13:00:00
1,1.22,0.0,2016-05-01 14:00:00
2,1.20,0.0,2016-05-01 15:00:00
3,1.20,0.0,2016-05-01 16:00:00
4,1.20,0.0,2016-05-01 17:00:00


In [60]:
flood = df[['time', 'value']]
flood.set_index('time', inplace=True)
flood.head()

,value
time,
2016-05-01 13:00:00,1.24
2016-05-01 14:00:00,1.22
2016-05-01 15:00:00,1.20
2016-05-01 16:00:00,1.20
2016-05-01 17:00:00,1.20


In [61]:
flood.index = flood.index.to_period('H')

In [62]:
flood.index

DatetimeIndex(['2016-05-01 13:00:00', '2016-05-01 14:00:00',
               '2016-05-01 15:00:00', '2016-05-01 16:00:00',
               '2016-05-01 17:00:00', '2016-05-01 18:00:00',
               '2016-05-01 19:00:00', '2016-05-01 20:00:00',
               '2016-05-01 21:00:00', '2016-05-01 22:00:00',
               ...
               '2020-09-02 17:00:00', '2020-09-02 18:00:00',
               '2020-09-02 19:00:00', '2020-09-02 20:00:00',
               '2020-09-02 21:00:00', '2020-09-02 22:00:00',
               '2020-09-02 23:00:00', '2020-09-03 00:00:00',
               '2020-09-03 01:00:00', '2020-09-03 02:00:00'],
              dtype='datetime64[ns]', name='time', length=79748, freq=None)

In [63]:
flood.head()

,value
time,
2016-05-01 13:00:00,1.24
2016-05-01 14:00:00,1.22
2016-05-01 15:00:00,1.20
2016-05-01 16:00:00,1.20
2016-05-01 17:00:00,1.20


In [64]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(flood, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
# model_fit = model.fit(disp=False)

/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/seokmin/workspace/azure_ml_test/.venv/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [65]:
flood.tail()

,value
time,
2020-09-02 22:00:00,6.02
2020-09-02 23:00:00,5.99
2020-09-03 00:00:00,6.01
2020-09-03 01:00:00,6.07
2020-09-03 02:00:00,6.17


In [66]:
import datetime

start = datetime.datetime(2020, 9, 3, 3)
end = datetime.datetime(2020, 10, 3, 2)

In [67]:
# model_fit.predict(start=start, end=end).plot()

In [68]:
from pmdarima.arima import auto_arima

In [69]:
model = auto_arima(y=flood)

In [ ]:
model.predict()

2020-09-05 14:00    6.015948
2020-09-05 15:00    6.040174
2020-09-05 16:00    6.028681
2020-09-05 17:00    6.019553
2020-09-05 18:00    6.012304
2020-09-05 19:00    6.006545
2020-09-05 20:00    6.001972
2020-09-05 21:00    5.998340
2020-09-05 22:00    5.995455
2020-09-05 23:00    5.993164
Freq: H, dtype: float64